In [1]:
import sys


no_of_cpu =8
max_cores =8
executor_mem = '56g'


Job_Name = 'Mayur : reconcilation contactibility cust_id'

from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests

# mesos_ip = 'mesos://10.35.12.205:5050'
mesos_ip = 'mesos://10.33.195.18:5050'
# mesos_ip = 'mesos://10.33.195.43:5050'
spark.stop() ############# NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL ###############################
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','120m')
conf.set('spark.sql.crossJoin.enabled', 'true')

# gpdb_env = 'prod'

# if gpdb_env=='uat':
#     gpdb_url = "jdbc:postgresql://10.33.195.103:5432/gpadmin"
#     gpdb_user = "gpspark"
#     gpdb_pass = "spark@456"
# else:
#     gpdb_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
#     gpdb_user = "gpspark"
#     gpdb_pass = "spark@456"



conf.set('spark.executor.heartbeatInterval','120s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx16g -Xms16g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
#23-11-2020
conf.set('spark.memory.offHeap.enabled','true')
conf.set('spark.memory.offHeap.size','16g')

conf.set('spark.sql.shuffle.partitions','80')
#conf.set('spark.sql.crossJoin.enabled', 'true')
# conf.set('es.nodes',es_nodes)
# conf.set('es.port',es_port)
# conf.set('es.nodes.wan.only','true')
# conf.set("spark.sql.autoBroadcastJoinThreshold",-1)
conf.set("spark.sql.broadcastTimeout",9000)
# config spark.scroll.keepalive("10m")
conf.set("spark.scroll.keepalive","120m")
#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
#conf.set('spark.num.executors','4')

# conf.set('spark.es.net.http.auth.user', es_user)
# conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)


# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


from pyspark.sql.window import *
from pyspark.sql.functions import row_number

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
17398,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-01-28 14:37:28.310231'

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# loading table as dataframe
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_progress(table_name,source):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime','records').filter(col('table_name')==table_name)\
            .filter(col('source')==source).filter(col('records')!=0)
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run)) MC:2018-11-01
        #ocrm 2017-12-28 icn:2019-04-05 10:58:49.291000  ICRM:2018-01-26 2023-07-29 00:00:00.000

    except Exception as E: 
        Max_last_run = datetime.datetime(2018,10,30, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
         "user": user_prod,
         "password": pwd_prod,
         "dbtable": "customermart.progress",
         "server.port":"1150-1170"}


progress_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.selectExpr('table_name','source','to_datetime','from_datetime')#\
#.filter(to_date(col('start_time'))==current_date())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
progress_df.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[table_name: string, source: string, to_datetime: timestamp, from_datetime: timestamp]

In [75]:
group_cols = ['table_name','source']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
progress_df1 = progress_df.groupby(group_cols).agg(max('to_datetime').alias('max_to_datetime')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
progress_df_new = progress_df.groupby(col('table_name')).agg(max('to_datetime').alias('max_to_datetime')) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
col_str=['source_system_name','source_system_customer_id', 'communication_sub_type','value','last_updated','activity_id']
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark", "server.port":"1150-1170",
"password": "spark@456","dbschema":"customermart",
"dbtable": "contactability"}
contact_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
         .selectExpr(col_str)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
progress_contact = progress_df_new.filter(col('table_name').isin('contactability')).collect()[0][1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
progress_contact

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2024, 1, 27, 0, 0)

In [81]:
progress_contact_mastercraft = progress_df1.filter(col('table_name').isin('contactability')).filter(col('source').isin('MASTERCRAFT')).collect()[0][2]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
progress_contact_mastercraft

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2024, 1, 27, 0, 0)

In [83]:
contact_df.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, communication_sub_type: string, value: string, last_updated: timestamp, activity_id: bigint]

In [84]:
contactability_cust_id = contact_df.filter(col('last_updated') <= progress_contact).select("source_system_customer_id").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
contactability_cust_id.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

88251871

In [86]:
contactability_cust_id_master = contact_df.filter(col('last_updated') <= progress_contact_mastercraft).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select("source_system_customer_id").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
contactability_cust_id_master.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [88]:
contactability_cust_id_master.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

87487443

In [89]:
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
# "dbschema": "public",
"dbtable": "public.reference_gc_genmst_intermediary"}

intermediary_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
                    .selectExpr('txt_mobile','txt_telephone','txt_telephone1','txt_contact_phone','txt_emer_person_mobile_number','txt_policy_phone','txt_alt_mobile','txt_email','txt_alt_email_id','txt_email_sms_reqd')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1160",
"dbschema": "staging",
"dbtable": "retention_auto_producer_contact"}
contact_prod = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr("contact as contact1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
intermediary_df.createOrReplaceTempView('intermediary_table')

query1_mob = """SELECT txt_mobile mobile from intermediary_table
union
select txt_telephone from intermediary_table
union
select txt_telephone1 from intermediary_table
union
select txt_contact_phone from intermediary_table
union
select txt_emer_person_mobile_number from intermediary_table
union
select txt_policy_phone from intermediary_table
union
select txt_alt_mobile from intermediary_table"""

intermediary_df_mob = sqlContext.sql(query1_mob)




gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1160",
"dbschema": "staging",
"dbtable": "retention_auto_producer_contact"}
contact_prod = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr("contact as contact1")

agent_no=intermediary_df_mob.union(contact_prod)



agent_no = agent_no.withColumn("mobile", when((length(col("mobile")) == 12), 
                                               regexp_replace(col("mobile"), "^91", "")).otherwise(lit(col("mobile"))))

agent_no = agent_no.withColumn("mobile", when((length(col("mobile")) == 11),
                                               regexp_replace(col("mobile"), "^0", "")).otherwise(lit(col("mobile"))))

agent_no.persist(pyspark.StorageLevel.MEMORY_AND_DISK)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[mobile: string]

In [92]:
query1_email = """select txt_email from intermediary_table
union
select txt_alt_email_id from intermediary_table
union
select txt_email_sms_reqd from intermediary_table"""


intermediary_df_email = sqlContext.sql(query1_email)


intermediary_df_email.persist(pyspark.StorageLevel.MEMORY_AND_DISK)


gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"server.port": "1151-1160",
"dbschema": "staging",
"dbtable": "retention_auto_producer_email"}

email_prod = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                    .selectExpr("email as email1")

agent_email=intermediary_df_email.union(email_prod)


agent_email.persist(pyspark.StorageLevel.MEMORY_AND_DISK)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[txt_email: string]

In [93]:
## to check for agent mobile doing left join contactability tbl with agent table with the below condition on contact number 

condition = [lower(trim(contact_df.value))==lower(trim(agent_no.mobile))]

contact_df_x1 = contact_df.join(agent_no,condition,how="left")
##contact_df_recheck_x1 = contact_df_recheck.filter(col('value').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
contact_df_x1.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, communication_sub_type: string, value: string, last_updated: timestamp, activity_id: bigint, mobile: string]

In [95]:
contact_df_x1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_system_name', 'source_system_customer_id', 'communication_sub_type', 'value', 'last_updated', 'activity_id', 'mobile']

In [96]:
## to check for agent email doing left join contactability tbl with agent table with the below condition on email id

conditions = [lower(trim(contact_df.value))==lower(trim(agent_email.txt_email))]

contact_df_x2 = contact_df_x1.join(agent_email,conditions,how="left")
##contact_df_recheck_x1 = contact_df_recheck.filter(col('value').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
contact_df_x2.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_system_name', 'source_system_customer_id', 'communication_sub_type', 'value', 'last_updated', 'activity_id', 'mobile', 'txt_email']

In [98]:
contact_df_x2.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, communication_sub_type: string, value: string, last_updated: timestamp, activity_id: bigint, mobile: string, txt_email: string]

In [99]:
contact_df_x2=contact_df_x2.filter(col('value').isNotNull()).filter(col('source_system_customer_id').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
## checking for agent mobile and email w.r.t 'value' column from contactability table
contact_df_x2_cnt = contact_df_x2.filter((contact_df_x2.value == contact_df_x2.mobile) | (contact_df_x2.value == contact_df_x2.txt_email)).filter(col('last_updated') <= progress_contact_mastercraft ).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select('source_system_customer_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'contact_df_x2' is not defined
Traceback (most recent call last):
NameError: name 'contact_df_x2' is not defined



In [101]:
contact_df_x2_cnt.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [102]:
contact_df_x2_cnt.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

66834

In [103]:
contact_df_x2_master = contact_df_x2.filter(col('last_updated') <= progress_contact_mastercraft ).filter(col('source_system_name').isin('GC','S3','CRM','NA','WB')).select('source_system_customer_id').distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
contact_df_x2_master.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

87487443

In [105]:
contact_df_x2_master.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [106]:
## creating new dataframe except agent contact details by using ExceptALL function

contact_df_x2_cnt_new_rechk = contact_df_x2_master.exceptAll(contact_df_x2_cnt)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
contact_df_x2_cnt_new_rechk.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [108]:
contact_df_x2_cnt_new_rechk.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

87420609

## Mastercraft 

In [110]:
contact_email_pri=contact.withColumn('activity_id', hash(col('email_primary')))
contact_email_pri=contact_email_pri.withColumn('verification_flag', when(trim(col('email_primary_is_verified')).isNull(), lit('G')).otherwise(lit('I')))
contact_email_pri=contact_email_pri.withColumn('verification_source', when(trim(col('email_primary_is_verified')).isNull(), lit(None)).otherwise(col('source_id')))
contact_email_pri=contact_email_pri.withColumn('value',col('email_primary'))\
# contact_email_pri=contact_email_pri.withColumn('value',when(col('email_primary').rlike(broadcast_regex.value),col('email_primary'))
#                                  .otherwise(lit(None)))

contact_email_pri = contact_email_pri.filter(~(lower(col("value")).contains("abc@")))\
.filter(~(lower(col("value")).contains("xyz@")))\
.filter(~(lower(col("value")).contains("1234@")))\
.filter(~(lower(col("value")).contains("null@")))\
.filter(~(lower(col("value")).contains("na@")))\
.filter(~(lower(col("value")).contains("00@")))


contact_email_pri=contact_email_pri.withColumn('dnd_flag', lit(None))
contact_email_pri=contact_email_pri.withColumn('communication_type', lit('email'))
contact_email_pri=contact_email_pri.withColumn('communication_sub_type', lit('email'))
contact_email_pri=contact_email_pri.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))
contact_email_pri=contact_email_pri.withColumn('contactability_json', lit(None))
contact_email_pri=contact_email_pri.withColumn('preference_flag', lit(None))
contact_email_pri=contact_email_pri.withColumn('date_reported', col('dc_action_date'))
contact_email_pri=contact_email_pri.withColumn('last_updated', col('dc_action_date'))
contact_email_pri=contact_email_pri.withColumn('activity_date', col('dc_action_date'))
contact_email_pri=contact_email_pri.withColumn('source_system_name', col('source_id'))
contact_email_pri=contact_email_pri.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
contact1=contact_email_pri.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_id').cast('string'),\
                         col('communication_type').cast('string'),col('communication_sub_type').cast('string'),col('activity_date').cast('timestamp'),\
                         col('value'),col('verification_flag').cast('string'),col('verification_source').cast('string'),col('date_reported').cast('timestamp'),col('last_updated').cast('timestamp'),col('preference_flag').cast('string'),col('dnd_flag').cast('string'),col('contactability_json'),\
                         col('primary_key'))
contact1=contact1.filter(col("value").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
contact_email_sec=contact.withColumn('activity_id', hash(col('email_secondary')))
contact_email_sec=contact_email_sec.withColumn('verification_flag', when(trim(col('email_secondary_is_verified')).isNull(), lit('G')).otherwise(lit('I')))
contact_email_sec=contact_email_sec.withColumn('verification_source', when(trim(col('email_secondary_is_verified')).isNull(), lit(None)).otherwise(col('source_id')))
# contact_email_sec=contact_email_sec.withColumn('value',when(col('email_secondary').rlike(broadcast_regex.value),col('email_secondary'))
#                   contact_email_sec        .otherwise(lit(None)))
contact_email_sec=contact_email_sec.withColumn('value',col('email_secondary'))

contact_email_sec = contact_email_sec.filter(~(lower(col("value")).contains("abc@")))\
.filter(~(lower(col("value")).contains("xyz@")))\
.filter(~(lower(col("value")).contains("1234@")))\
.filter(~(lower(col("value")).contains("null@")))\
.filter(~(lower(col("value")).contains("na@")))\
.filter(~(lower(col("value")).contains("00@")))

contact_email_sec=contact_email_sec.withColumn('dnd_flag', lit(None))
contact_email_sec=contact_email_sec.withColumn('communication_type', lit('email'))
contact_email_sec=contact_email_sec.withColumn('communication_sub_type', lit('email'))
contact_email_sec=contact_email_sec.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))
contact_email_sec=contact_email_sec.withColumn('contactability_json', lit(None))
contact_email_sec=contact_email_sec.withColumn('preference_flag', lit(None))
contact_email_sec=contact_email_sec.withColumn('date_reported', col('dc_action_date'))
contact_email_sec=contact_email_sec.withColumn('last_updated', col('dc_action_date'))
contact_email_sec=contact_email_sec.withColumn('activity_date', col('dc_action_date'))
contact_email_sec=contact_email_sec.withColumn('source_system_name', col('source_id'))
contact_email_sec=contact_email_sec.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
contact2=contact_email_sec.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_id').cast('string'),\
                         col('communication_type').cast('string'),col('communication_sub_type').cast('string'),col('activity_date').cast('timestamp'),\
                         col('value'),col('verification_flag').cast('string'),col('verification_source').cast('string'),col('date_reported').cast('timestamp'),col('last_updated').cast('timestamp'),col('preference_flag').cast('string'),col('dnd_flag').cast('string'),col('contactability_json'),\
                         col('primary_key'))
contact2=contact2.filter(col("value").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
contact_f1 = contact1.union(contact2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
contact_mob_pri=contact.withColumn('activity_id', hash(col('mobile_primary')))
contact_mob_pri=contact_mob_pri.withColumn('verification_flag', when(col('mobile_primary_is_verified').isNull(), lit('G')).otherwise(lit('N')))
contact_mob_pri=contact_mob_pri.withColumn('verification_source', when(col('mobile_primary_is_verified').isNull(), lit(None)).otherwise(col('source_id')))
contact_mob_pri=contact_mob_pri.withColumn('value',col('mobile_primary'))

contact_mob_pri=contact_mob_pri.withColumn('value',when(col('mobile_primary').rlike('^(?:(?:\+|0{0,2})91(\s*[\-]\s*)?|[0]?)?[6-9]\d{9}$'),col('mobile_primary'))
                       .otherwise(lit(None)))

contact_mob_pri = contact_mob_pri.withColumn("value", when((length(col("value")) == 12), 
                                       regexp_replace(col("value"), "^91", "")).otherwise(lit(col("value"))))

contact_mob_pri = contact_mob_pri.withColumn("value", when((length(col("value")) == 11),
                                      regexp_replace(col("value"), "^0", "")).otherwise(lit(col("value"))))


contact_mob_pri=contact_mob_pri.withColumn('value',when(col('value').rlike("^(?!(\d)\1{8})\d{10}$"),col('value'))
                          .otherwise(lit(None)))
contact_mob_pri=contact_mob_pri.withColumn('dnd_flag', when(trim(col('mobile_primary_dnd'))=='1', lit('1')).otherwise(lit(None)))
contact_mob_pri=contact_mob_pri.withColumn('communication_type', lit('phone'))
contact_mob_pri=contact_mob_pri.withColumn('communication_sub_type', lit('mobile'))
contact_mob_pri=contact_mob_pri.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))
contact_mob_pri=contact_mob_pri.withColumn('contactability_json', lit(None))
contact_mob_pri=contact_mob_pri.withColumn('preference_flag', lit(None))
contact_mob_pri=contact_mob_pri.withColumn('date_reported', col('dc_action_date'))
contact_mob_pri=contact_mob_pri.withColumn('last_updated', col('dc_action_date'))
contact_mob_pri=contact_mob_pri.withColumn('activity_date', col('dc_action_date'))
contact_mob_pri=contact_mob_pri.withColumn('source_system_name', col('source_id'))
contact_mob_pri=contact_mob_pri.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
contact3=contact_mob_pri.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_id').cast('string'),\
                         col('communication_type').cast('string'),col('communication_sub_type').cast('string'),col('activity_date').cast('timestamp'),\
                         col('value'),col('verification_flag').cast('string'),col('verification_source').cast('string'),col('date_reported').cast('timestamp'),col('last_updated').cast('timestamp'),col('preference_flag').cast('string'),col('dnd_flag').cast('string'),col('contactability_json'),\
                         col('primary_key'))
contact3=contact3.filter(col("value").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
contact_f2 = contact_f1.union(contact3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
contact_mob_sec=contact.withColumn('activity_id', hash(col('mobile_secondary')))
contact_mob_sec=contact_mob_sec.withColumn('verification_flag',when(col('mobile_secondary').isNotNull(), lit('G')))
contact_mob_sec=contact_mob_sec.withColumn('verification_source',when(col('mobile_secondary').isNotNull(), lit(None)))
contact_mob_sec=contact_mob_sec.withColumn('value',col('mobile_secondary'))
contact_mob_sec=contact_mob_sec.withColumn('value',when(col('mobile_secondary').rlike('^(?:(?:\+|0{0,2})91(\s*[\-]\s*)?|[0]?)?[6-9]\d{9}$'),col('mobile_secondary'))
                                 .otherwise(lit(None)))

contact_mob_sec = contact_mob_sec.withColumn("value", when((length(col("value")) == 12), 
                                               regexp_replace(col("value"), "^91", "")).otherwise(lit(col("value"))))

contact_mob_sec = contact_mob_sec.withColumn("value", when((length(col("value")) == 11),
                                               regexp_replace(col("value"), "^0", "")).otherwise(lit(col("value"))))

contact_mob_sec=contact_mob_sec.withColumn('value',when(col('value').rlike("^(?!(\d)\1{8})\d{10}$"),col('value'))
                                 .otherwise(lit(None)))
contact_mob_sec=contact_mob_sec.withColumn('dnd_flag', when(trim(col('mobile_secondary_dnd'))=='1', lit('1')).otherwise(lit(None)))
contact_mob_sec=contact_mob_sec.withColumn('communication_type', lit('phone'))
contact_mob_sec=contact_mob_sec.withColumn('communication_sub_type', lit('mobile'))
contact_mob_sec=contact_mob_sec.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))
contact_mob_sec=contact_mob_sec.withColumn('contactability_json', lit(None))
contact_mob_sec=contact_mob_sec.withColumn('preference_flag', lit(None))
contact_mob_sec=contact_mob_sec.withColumn('date_reported', col('dc_action_date'))
contact_mob_sec=contact_mob_sec.withColumn('last_updated', col('dc_action_date'))
contact_mob_sec=contact_mob_sec.withColumn('activity_date', col('dc_action_date'))
contact_mob_sec=contact_mob_sec.withColumn('source_system_name', col('source_id'))
contact_mob_sec=contact_mob_sec.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
contact4=contact_mob_sec.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_id').cast('string'),\
                         col('communication_type').cast('string'),col('communication_sub_type').cast('string'),col('activity_date').cast('timestamp'),\
                         col('value'),col('verification_flag').cast('string'),col('verification_source').cast('string'),col('date_reported').cast('timestamp'),col('last_updated').cast('timestamp'),col('preference_flag').cast('string'),col('dnd_flag').cast('string'),col('contactability_json'),\
                         col('primary_key'))
contact4=contact4.filter(col("value").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
contact_f3= contact_f2.union(contact4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
contact_land=contact.filter(col("incominglandlinehome").isNotNull())
contact_land=contact_land.withColumn('activity_id', hash(col('incominglandlinehome')))
contact_land=contact_land.withColumn('verification_flag', when(col('landline_home_is_verified').isNull(), lit('G')).otherwise(lit('N')))
contact_land=contact_land.withColumn('verification_source', when(col('landline_home_is_verified').isNull(), lit(None)).otherwise(col('source_id')))

# contact_land=contact_land.withColumn('value',col('incominglandlinehome'))
contact_land = contact_land.withColumn("value",regexp_replace(col("incominglandlinehome"), " ", ""))

contact_land=contact_land.withColumn('value',when(col('value').rlike('^(?:(?:\+|0{0,2})91(\s*[\-\s]\s*)?|[0]?)?[2-9]?\d{1,3}(\s*[\-\s]\s*)?\d{6,8}$'),col('value'))
                                 .otherwise(lit(None)))



contact_land=contact_land.withColumn('dnd_flag', lit(None))
contact_land=contact_land.withColumn('communication_type', lit('phone'))
contact_land=contact_land.withColumn('communication_sub_type', lit('landline'))
contact_land=contact_land.withColumn('source_system_customer_id', coalesce('source_customer_id','dc_unified_id'))
contact_land=contact_land.withColumn('contactability_json', lit(None))
contact_land=contact_land.withColumn('preference_flag', lit(None))
contact_land=contact_land.withColumn('date_reported', col('dc_action_date'))
contact_land=contact_land.withColumn('last_updated', col('dc_action_date'))
contact_land=contact_land.withColumn('activity_date', col('dc_action_date'))
contact_land=contact_land.withColumn('source_system_name', col('source_id'))
contact_land=contact_land.withColumn('primary_key', concat_ws('',trim(col('source_id')),trim(col('source_system_customer_id')), trim(col('activity_id'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
contact5=contact_land.select(col('source_system_name').cast('string'),col('source_system_customer_id').cast('string'),col('activity_id').cast('string'),\
                         col('communication_type').cast('string'),col('communication_sub_type').cast('string'),col('activity_date').cast('timestamp'),\
                         col('value'),col('verification_flag').cast('string'),col('verification_source').cast('string'),col('date_reported').cast('timestamp'),col('last_updated').cast('timestamp'),col('preference_flag').cast('string'),col('dnd_flag').cast('string'),col('contactability_json'),\
                         col('primary_key'))
contact5=contact5.filter(col("value").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
contact_f4= contact_f3.union(contact5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
# df_masterCraft= contact_f4.withColumn('contactability_json', to_json(lit(None)))
df_masterCraft=contact_f4.withColumn('last_updated', col('date_reported'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
from pyspark.sql.window import Window

df_masterCraft = df_masterCraft.withColumn("rn", F.row_number()
        .over(Window.partitionBy("primary_key")
        .orderBy(F.col("last_updated").desc())))

df_masterCraft = df_masterCraft.filter(F.col("rn") == 1).drop("rn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
df_masterCraft=df_masterCraft.select(['source_system_name','source_system_customer_id','activity_id','activity_date','communication_type','communication_sub_type','value','verification_flag','verification_source','date_reported','last_updated','preference_flag','contactability_json','dnd_flag'])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
df_masterCraft = df_masterCraft.filter(~(col("value").isin(['1234567890','9876543210','9999988888','9090909090','9000000000','9900000000'])))
df_masterCraft = df_masterCraft.filter(~(lower(trim(col("value"))).isin(['abc@gmail.com','xyz@gmail.com','123@gmail.com','abc@d.com','abcd@gmail.com','null'])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
df_masterCraft.repartition('value')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, activity_id: string, activity_date: timestamp, communication_type: string, communication_sub_type: string, value: string, verification_flag: string, verification_source: string, date_reported: timestamp, last_updated: timestamp, preference_flag: string, contactability_json: null, dnd_flag: string]

In [129]:
condition = [lower(trim(df_masterCraft.value))==lower(trim(agent_no.mobile))]

df_masterCraft = df_masterCraft.join(agent_no,condition,how="left")
df_masterCraft = df_masterCraft.filter(col('mobile').isNull()).drop(df_masterCraft.mobile)

condition = [lower(trim(df_masterCraft.value))==lower(trim(agent_email.txt_email))]

df_masterCraft = df_masterCraft.join(agent_email,condition,how="left")
df_masterCraft = df_masterCraft.filter(col('txt_email').isNull()).drop(df_masterCraft.txt_email)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
df_masterCraft=df_masterCraft.filter(col('value').isNotNull()).filter(col('source_system_customer_id').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
df_masterCraft.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_name: string, source_system_customer_id: string, activity_id: string, activity_date: timestamp, communication_type: string, communication_sub_type: string, value: string, verification_flag: string, verification_source: string, date_reported: timestamp, last_updated: timestamp, preference_flag: string, contactability_json: null, dnd_flag: string]

In [132]:
df_masterCraft_cnt = df_masterCraft.filter(col('last_updated') <= 'progress_contact_mastercraft').select("source_system_customer_id").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
df_masterCraft_cnt.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [134]:
df_masterCraft_cnt.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

87498932

In [135]:
## Checking customer id which are in mastercraft table but not in contactability table with exceptall function (after removing agent details fro both tables : mastercraft and contactability tbls) 

final_result = df_masterCraft_cnt.exceptAll(contact_df_x2_cnt_new_rechk)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
final_result.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[source_system_customer_id: string]

In [137]:
final_result.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

78323

In [138]:
final_result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|source_system_customer_id|
+-------------------------+
|               6054209340|
|               6077444098|
|               6080041326|
|               6138645981|
|               6149522131|
|               6150253007|
|               6155297929|
|               6155300278|
|               6155303529|
|               6155313601|
|               6155319711|
|               6155327628|
|               6155336421|
|               6155342131|
|               6076431830|
|               6007147191|
|               6155300138|
|               6155322729|
|               6155337943|
|               6155340651|
+-------------------------+
only showing top 20 rows

In [2]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…